TODO
1. [x] remove exact duplicates
2. [x] remove artists, albumname, trackname column (assuming artists doesnt explain danceability)
3. [x] duration ms to seconds - float

In [1]:
# importing libraries
import pandas as pd

In [2]:
# importing the spotify song info dataset
df = pd.read_csv('../data/raw/spotify.csv')

In [3]:
# drop duplicates
df = df.drop_duplicates()

In [4]:
# removing 3 columns
df = df.drop(columns=['artists', 'album_name', 'track_name'])

In [5]:
df.columns

Index(['track_id', 'popularity', 'duration_ms', 'explicit', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [6]:
# converting ms to s
df['duration_ms'] = df['duration_ms']/1000
df.rename(columns = {'duration_ms':'duration_s'}, inplace = True)

In [7]:
# grouping by and imploding track_genre col
def join_genres(genres):
    return ','.join(genres)

agg_dict = {
    'track_genre': join_genres,
    'popularity': 'first',
    'duration_s': 'first',
    'explicit': 'first',
    'danceability': 'first',
    'energy': 'first',
    'key': 'first',
    'loudness': 'first',
    'mode': 'first',
    'speechiness': 'first',
    'acousticness': 'first',
    'instrumentalness': 'first',
    'liveness': 'first',
    'valence': 'first',
    'tempo': 'first',
    'time_signature': 'first'
}

# Group by 'track_id' and apply the aggregation using the agg_dict
result_df = df.groupby('track_id', as_index=False).agg(agg_dict)

In [8]:
result_df['explicit'] = result_df['explicit'].apply(lambda x: 1 if x else 0)

In [9]:
result_df

,track_id,track_genre,popularity,duration_s,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0000vdREvCVMxbQTkS888c,german,44,160.725,1,0.910,0.37400,8,-9.844,0,0.1990,0.075700,0.00301,0.1540,0.432,104.042,4
1,000CC8EParg64OmTxVnZ0p,club,47,322.933,0,0.269,0.51600,0,-7.361,1,0.0366,0.406000,0.00000,0.1170,0.341,178.174,4
2,000Iz0K615UepwSJ5z2RE5,minimal-techno,22,515.360,0,0.686,0.56000,5,-13.264,0,0.0462,0.001140,0.18100,0.1110,0.108,119.997,4
3,000RDCYioLteXcutOjeweY,hip-hop,62,190.203,0,0.679,0.77000,0,-3.537,1,0.1900,0.058300,0.00000,0.0825,0.839,161.721,4
4,000qpdoc97IMTBvF8gwcpy,minimal-techno,19,331.240,0,0.519,0.43100,6,-13.606,0,0.0291,0.000964,0.72000,0.0916,0.234,129.971,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88890,7zxHiMmVLt4LGWpOMqOpUh,pop-film,56,325.156,0,0.766,0.38200,7,-11.464,0,0.0324,0.698000,0.00143,0.1570,0.672,119.992,4
88891,7zxpdh3EqMq2JCkOI0EqcG,disney,23,109.573,0,0.529,0.00879,10,-32.266,1,0.0587,0.996000,0.95900,0.0916,0.510,82.694,4
88892,7zyYmIdjqqiX6kLryb7QBx,mandopop,61,260.573,0,0.423,0.36000,3,-9.458,1,0.0372,0.728000,0.00000,0.1050,0.291,130.576,4
88893,7zybSU9tFO9HNlwmGF7stc,electronic,54,234.300,0,0.649,0.83400,10,-11.430,0,0.0397,0.268000,0.93200,0.0974,0.150,125.004,4


In [10]:
result_df.to_csv('../data/processed/cleaned_data_imploded.csv', index=0)

In [11]:
# explode result_df
exploded_res = result_df.copy()
exploded_res['track_genre'] = exploded_res['track_genre'].str.split(',')
exploded_res = exploded_res.explode('track_genre')

In [12]:
exploded_res.to_csv('../data/processed/cleaned_data_exploded.csv', index=0)